In [1]:
from sqlalchemy import create_engine
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import statsmodels.formula.api as sm
import time
import csv
from datetime import timedelta
%matplotlib inline

In [2]:
df1 = pd.read_csv("rt_trips_full.csv", sep=',')
df1 = df1.drop(['datasource', 'basin', 'tenderlot', 'suppressed', 'justificationid', 'lastupdate', 'note'], axis=1)
df1.shape

(2041415, 9)

In [3]:
df1['dayofservice'] = pd.to_datetime(df1['dayofservice'])

In [6]:
# Add school holiday column, 0 will mean NOT in a school holiday
df1['school_hol'] = 0
df1.head(5)

,dayofservice,tripid,lineid,routeid,direction,plannedtime_arr,plannedtime_dep,actualtime_arr,actualtime_dep,school_hol
0,2016-02-09,2824642,145,145_105,2,45668,40800,45375.0,40782.0,0
1,2016-02-09,2826717,9,9_7,2,65553,60600,66258.0,60582.0,0
2,2016-02-09,2826730,54A,54A_12,2,65951,62100,66053.0,62078.0,0
3,2016-02-09,2826743,7,7_51,1,54763,50400,54854.0,50383.0,0
4,2016-02-09,2812908,39,39_20,1,27375,22920,27318.0,22931.0,0


In [4]:
school_holidays = ['2016-01-01','2016-01-04','2016-01-05','2016-02-15','2016-02-16','2016-02-17','2016-02-18','2016-02-19','2016-02-19','2016-03-16','2016-03-17','2016-03-18','2016-03-21','2016-03-22','2016-03-23','2016-03-24','2016-03-25','2016-03-28','2016-03-29','2016-03-30','2016-03-31','2016-04-01','2016-06-06','2016-06-07','2016-06-08','2016-06-09','2016-06-10','2016-06-13','2016-06-14','2016-06-15','2016-06-16','2016-06-17','2016-06-20','2016-06-21','2016-06-22','2016-06-23','2016-06-24','2016-06-27','2016-06-28','2016-06-29','2016-06-30','2017-01-02','2017-01-03','2017-01-04','2017-01-05','2017-01-06','2017-02-20','2017-02-21','2017-02-22','2017-02-23','2017-02-24','2017-04-09','2017-04-10','2017-04-11','2017-04-12','2017-04-13','2017-04-14','2017-04-16','2017-04-17','2017-04-18','2017-04-19','2017-04-20','2017-04-21','2017-06-05','2017-06-06','2017-06-07','2017-06-08','2017-06-09','2017-06-12','2017-06-13','2017-06-14','2017-06-15','2017-06-16','2017-06-19','2017-06-20','2017-06-21','2017-06-22','2017-06-23','2017-06-26','2017-06-27','2017-06-28','2017-06-29','2017-06-30']
print(len(school_holidays))

83


In [18]:

for i in school_holidays:
    df1.loc[df1['dayofservice'] == i, 'school_hol'] = 1

In [19]:
# Check if a school holiday has flag 1

check = df1['dayofservice'] == '2017-02-22'
check = df1[check]
check.head(5)

,dayofservice,tripid,lineid,routeid,direction,plannedtime_arr,plannedtime_dep,actualtime_arr,actualtime_dep,school_hol
1001099,2017-02-22,4381744,4,4_12,2,57516,53100,58214.0,53136.0,1
1001100,2017-02-22,4388860,84A,84A_1,1,67967,65400,68335.0,65407.0,1
1001101,2017-02-22,4388888,39A,39A_43,2,65671,60600,65991.0,60552.0,1
1001102,2017-02-22,4388924,25X,25X_5,2,31800,27300,31963.0,NaN,1
1001103,2017-02-22,4389028,39,39_21,2,63799,58920,64065.0,58926.0,1


In [20]:
# Check if a normal weekday has flag 0

check2 = df1['dayofservice'] == '2016-04-04'
check2 = df1[check2]
check2.head(5)

,dayofservice,tripid,lineid,routeid,direction,plannedtime_arr,plannedtime_dep,actualtime_arr,actualtime_dep,school_hol
415405,2016-04-04,3007057,11,11_42,2,36236,31200,36401.0,31213.0,0
415406,2016-04-04,3018280,11,11_42,2,38936,33900,38517.0,33946.0,0
415407,2016-04-04,3008751,11,11_42,2,33836,28800,34706.0,28800.0,0
415408,2016-04-04,3010395,11,11_42,2,40436,35400,40968.0,35384.0,0
415409,2016-04-04,3004450,40,40_24,2,80959,76800,82504.0,76692.0,0
